I am going to attempt to group images by facial alignment, in the hopes of making PCA easier, and hopefully to get better results from our PCA. With these groupings, perhaps we perform PCA on each group individually.

The following function should use libraries, Dlib and Mediapipe to detect the facial landmarks in our images. It will be interesting to ee 

In [1]:
# import packages
import numpy as np
import scipy.io as sio
import matplotlib
import matplotlib.pyplot as plt
from numpy.matlib import repmat
from sklearn.preprocessing import normalize
import pandas as pd

In [11]:
# import dataset

# import additional packages to process images as dataset
import os
from PIL import Image

# wrangle images and transform into arrays
def load_images_from_folder(folder_path, target_size=(75, 75), color_mode='RGB'):
    # initialize empty list
    images = []
    # loop through images in folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            # Load image
            img = Image.open(file_path).convert(color_mode)  # Convert to grayscale
            img = img.resize(target_size)  # Resize to target size
            images.append(np.array(img))
            # images.append(np.array(img).flatten())  # Flatten to 1D array
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
    return np.array(images)

# Load dataset
folder_path = "simpsons_faces_kostastokis/cropped"
simpsons_df = load_images_from_folder(folder_path, target_size=(75,75))

# Check the shape of the dataset
print(f"Dataset shape: {simpsons_df.shape}")


Dataset shape: (9877, 75, 75, 3)


In [17]:

import cv2
import dlib
import numpy as np

# set up detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# create function to estimate angle
def estimate_face_angle(image):
    rects = detector(image, 1)
    for rect in rects:
        shape = predictor(image, rect)
        landmarks = np.array([[p.x, p.y] for p in shape.parts()])
        # Calculate angle between eyes, could try other things as well
        left_eye = np.mean(landmarks[36:42], axis=0)
        right_eye = np.mean(landmarks[42:48], axis=0)
        angle = np.arctan2(right_eye[1] - left_eye[1], right_eye[0] - left_eye[0]) * (180.0 / np.pi)
        return angle
    return None


In [18]:
# initialize empty list for angles of each image
angles = []

for img in simpsons_df:
    angle = estimate_face_angle(img)
    if angle is not None:
        angles.append(angle)
    else:
        angles.append('NaN')

In [20]:
print(angles)

['NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN'

Use Pose estimation / pre-trained models to estimate had angles, including horizontal rotation, vertical tilt, and tilt sideways. 

Group the images by face angle.

In [19]:
from collections import defaultdict

grouped_images = defaultdict(list)

for img, angle in zip(img, angles):
    # left profile
    if angle < -30: 
        grouped_images['left_profile'].append(img)
    # right profile
    elif angle > 30: 
        grouped_images['right_profile'].append(img)
    # front
    else:
        grouped_images['frontal'].append(img)


TypeError: '<' not supported between instances of 'str' and 'int'

Apply PCA to each group:

In [ ]:
for group, imgs in grouped_images.items():
    mean_simp = np.mean(imgs, axis=0)
    A = imgs - mean_simp
    U, S, Vt = np.linalg.svd(A, full_matrices=False)